<a href="https://colab.research.google.com/github/amazingashis/Hackfest/blob/master/pneumonia_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_recall_curve, roc_curve, accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import seaborn as sns 
plt.style.use('fivethirtyeight')
import pickle 
import os 
import numpy as np
import cv2 
%matplotlib inline
import pandas.util.testing as tm

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [2]:
labels = ['yes', 'no']
img_size = 200
def get_training_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [4]:
data = get_training_data('/content/drive/MyDrive/Data/data/Pneumonia data')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [5]:
X = []
y = []

for feature, label in data:
    X.append(feature)
    y.append(label)



# resize data for deep learning 
X = np.array(X).reshape(-1, img_size, img_size, 1)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

In [6]:
X_train = X_train / 255
X_test = X_test / 255

In [7]:
# good for balancing out disproportions in the dataset 
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=90, 
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=True,  
        vertical_flip=True)  

datagen.fit(X_train)

In [8]:
model = keras.Sequential([
  keras.layers.Conv2D(input_shape=(200,200,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  keras.layers.Flatten(),
  keras.layers.Dense(2, activation=tf.nn.softmax, name='Softmax')
])
model.summary()

testing = False
epochs = 15

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)

test_loss, test_acc = model.evaluate(X_test, y_test)
print('\nTest accuracy: {}'.format(test_acc))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 99, 99, 8)         80        
_________________________________________________________________
flatten (Flatten)            (None, 78408)             0         
_________________________________________________________________
Softmax (Dense)              (None, 2)                 156818    
Total params: 156,898
Trainable params: 156,898
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
7/7 [==============================] - 1s 60ms/step - loss: 2.9225 - accuracy: 0.4757
Epoch 2/15
7/7 [==============================] - 0s 64ms/step - loss: 1.3008 - accuracy: 0.6004
Epoch 3/15
7/7 [==============================] - 0s 62ms/step - loss: 0.9495 - accuracy: 0.5731
Epoch 4/15
7/7 [==============================] - 0s 63ms/step - loss: 0.3322 - accuracy:

In [9]:
print(model.predict(X_test))

[[9.9277145e-01 7.2285333e-03]
 [9.8830914e-01 1.1690883e-02]
 [9.3892235e-03 9.9061078e-01]
 [1.2624229e-01 8.7375772e-01]
 [9.8559040e-01 1.4409636e-02]
 [9.9872643e-01 1.2735904e-03]
 [9.6424586e-01 3.5754211e-02]
 [1.9227372e-01 8.0772626e-01]
 [9.9802291e-01 1.9770768e-03]
 [9.9938142e-01 6.1850081e-04]
 [5.7464768e-03 9.9425346e-01]
 [1.9364014e-01 8.0635983e-01]
 [8.2349833e-03 9.9176496e-01]
 [1.4651809e-02 9.8534817e-01]
 [1.9703674e-01 8.0296320e-01]
 [2.1967201e-01 7.8032804e-01]
 [9.7292028e-02 9.0270799e-01]
 [9.8942530e-01 1.0574654e-02]
 [7.7791232e-01 2.2208774e-01]
 [6.7117775e-04 9.9932885e-01]
 [9.4765328e-02 9.0523469e-01]
 [9.7338617e-01 2.6613867e-02]
 [9.5818961e-01 4.1810419e-02]
 [4.2688924e-01 5.7311082e-01]
 [9.9379063e-02 9.0062100e-01]
 [1.0757528e-02 9.8924249e-01]
 [6.1357749e-01 3.8642251e-01]
 [9.8139441e-01 1.8605564e-02]
 [1.0107853e-02 9.8989213e-01]
 [9.9695444e-01 3.0455161e-03]
 [6.9393694e-02 9.3060631e-01]
 [1.9841651e-03 9.9801588e-01]
 [3.5375

In [3]:
!pip install tensorflowjs

     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 112kB 7.8MB/s 
  Found existing installation: tensorflow-hub 0.11.0
    Uninstalling tensorflow-hub-0.11.0:
      Successfully uninstalled tensorflow-hub-0.11.0


In [10]:
import time
saved_model_path = "tmp/{}.h5".format(int(time.time()))

model.save(saved_model_path)

In [11]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./

2021-02-20 15:42:26.004681: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [12]:
import time
saved_model_path = "/tmp/{}".format(int(time.time()))
model.save(saved_model_path)

INFO:tensorflow:Assets written to: /tmp/1613835783/assets


In [ ]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./